### Homework: Implement possibilistic k-means

Goal:
1. Implement the mahalanobis_distance function.
2. Implement the calculate_eta function.
3. Implement the calculate_u.

Hint: the assignation matrix should not be set to zeros at the beginning. The equations can be found in the slides. 

**Deadline:** 4.04.2018

## Importing libraries

In [7]:
import numpy as np
import math
import scipy.spatial.distance as sc_dist
import random

## Introducing and preprocessing aircraft dataset

In [8]:
# Introducing familiar aircraft database
X=np.array([(4,1940),(9,2960),(9,4630),(78,1528),(90,2040),(50,3700),(467,14815),(509,15200),(290,15700),(215,6045)])
# Preprocessing (normalizing) dataset
train_data = np.array(X)
max_values = train_data.max(0)
X_norm = np.divide(train_data,max_values)
# Calcualting mean value for each dimension
mean_vec = np.mean(X_norm, axis=0)
# Estimating covariance matrix for characteristics
cov_mat = (X_norm - mean_vec).T.dot((X_norm - mean_vec)) / (X_norm.shape[0]-1)
# Inverting matrix on purpose of Mahalanobis distance calculation
inv_cov_mat=np.linalg.inv(cov_mat)

## Defining mah_dist_func()

In [9]:
def mah_dist_func(x,v,inv_cov_mat):
    diff_vect=np.subtract(x,v)
    cov_vect_mult=np.dot(inv_cov_mat,diff_vect)
    return math.sqrt(np.dot(diff_vect.T,cov_vect_mult))

l=[]
for i in range(len(X_norm)):
    l_temp=[]
    for j in range(len(X_norm)):
        l_temp.append(mah_dist_func(X_norm[i],X_norm[j],inv_cov_mat))
    l.append(list(l_temp))
mah_matrix=np.array(l)
print (mah_matrix)

[[0.         0.35432283 1.02523766 1.07854471 1.03960193 0.30376842
  2.42610079 2.67559252 2.73085546 1.33368116]
 [0.35432283 0.         0.67154923 1.40706243 1.35827325 0.27281073
  2.43795203 2.726312   2.42712122 1.54083292]
 [1.02523766 0.67154923 0.         2.06330978 2.00722105 0.86686098
  2.64224371 2.98389969 1.93636921 2.07431507]
 [1.07854471 1.40706243 2.06330978 0.         0.0937244  1.20056146
  2.2364494  2.33559968 3.47228587 0.76300522]
 [1.03960193 1.35827325 2.00722105 0.0937244  0.         1.14142418
  2.15083773 2.25756895 3.38651361 0.68166885]
 [0.30376842 0.27281073 0.86686098 1.20056146 1.14142418 0.
  2.2002277  2.4753167  2.44604028 1.27083342]
 [2.42610079 2.43795203 2.64224371 2.2364494  2.15083773 2.2002277
  0.         0.39029478 2.54210423 1.47696848]
 [2.67559252 2.726312   2.98389969 2.33559968 2.25756895 2.4753167
  0.39029478 0.         2.9238253  1.57690171]
 [2.73085546 2.42712122 1.93636921 3.47228587 3.38651361 2.44604028
  2.54210423 2.9238253

Mahalanobis distance could be also calculated using function from scipy library: cdist(XA, XB, metric='mahalanobis', VI='inverse_cov_mat')

In [10]:
print (sc_dist.cdist(X_norm, X_norm, metric='mahalanobis', VI=inv_cov_mat))

[[0.         0.35432283 1.02523766 1.07854471 1.03960193 0.30376842
  2.42610079 2.67559252 2.73085546 1.33368116]
 [0.35432283 0.         0.67154923 1.40706243 1.35827325 0.27281073
  2.43795203 2.726312   2.42712122 1.54083292]
 [1.02523766 0.67154923 0.         2.06330978 2.00722105 0.86686098
  2.64224371 2.98389969 1.93636921 2.07431507]
 [1.07854471 1.40706243 2.06330978 0.         0.0937244  1.20056146
  2.2364494  2.33559968 3.47228587 0.76300522]
 [1.03960193 1.35827325 2.00722105 0.0937244  0.         1.14142418
  2.15083773 2.25756895 3.38651361 0.68166885]
 [0.30376842 0.27281073 0.86686098 1.20056146 1.14142418 0.
  2.2002277  2.4753167  2.44604028 1.27083342]
 [2.42610079 2.43795203 2.64224371 2.2364494  2.15083773 2.2002277
  0.         0.39029478 2.54210423 1.47696848]
 [2.67559252 2.726312   2.98389969 2.33559968 2.25756895 2.4753167
  0.39029478 0.         2.9238253  1.57690171]
 [2.73085546 2.42712122 1.93636921 3.47228587 3.38651361 2.44604028
  2.54210423 2.9238253

**Both implementations lead to identical results**

In [28]:
# Caluclation of eta radius = calculate_eta_function
def calc_eta_radius(centers,ass_matrix,X_norm,m=2):
    eta_list=[]
    for i in range(0,ass_matrix.shape[1]):
        numerator=0
        denominator=0
        for j in range(len(X_norm)):
            #print (j,i)
            numerator+=ass_matrix[j][i]**m*mah_dist_func(centers[i],X_norm[j],inv_cov_mat)**2
            denominator+=ass_matrix[j][i]**m
        eta_i = numerator/denominator
        eta_list.append(eta_i)
    return eta_list

def calculate_u(x,i,eta_list,m=2):
    global centers
    dist=mah_dist_func(centers[i],x,inv_cov_mat)
    val=(1+(dist/eta_list[i])**(2/(m-1)))**(-1)
    #1/(1+(dist**2/eta_list[i])**(1/(m-1)))
    return val

centers = []

data_set=np.sort(X_norm,axis=0)
#print (data_set)
groups = 2
space=[[0,1],[0,1]]

error_margin = 0.01
m = 2.0

#Script for generation of initial assignation matrix
def init_assign(X_norm,groups):
    #dummy_data=np.array(X_norm)
    #sorted_data=np.sort(dummy_data)
    row_num,col_num=X_norm.shape
    segment=int(row_num/groups)
    #segment=int(row_num)
    ass_matrix=[]
    for i in range(0,col_num):
        n=0
        ntot=0
        while n<segment and ntot<row_num:
            l_zero=np.zeros(col_num)
            l_zero[i]=1
            ass_matrix.append(l_zero)
            ntot+=1
            n+=1
    ass_matrix=np.array(ass_matrix)  
    #print (ass_matrix.shape[1])
    return ass_matrix

assignation=init_assign(X_norm,groups)
calculate_new_centers(assignation)

def calculate_new_centers(u):
    global centers
    new_centers=[]
    for c in range(groups):
        u_x_vector=np.zeros(2)
        u_scalar=0.0
        for i in range(len(data_set)):
            # u_scalar is a variable for calculating normalization factor
            u_scalar = u_scalar+(u[i][c]**m)
            # u_x is a variable for calculating displacement of group center. M is =2. If vector is assigned to cluster, move 
            # current center by u_x_vector*assignment_value. 
            u_x_vector=np.add(u_x_vector,np.multiply(u[i][c]**m,data_set[i]))
        new_centers.append(np.divide(u_x_vector,u_scalar))
    centers=new_centers

def calculate_differences(new_assignation):
    global assignation    
    return np.sum(np.abs(np.subtract(assignation,new_assignation)))

#Redefining cluster function
def cluster():
    global assignation    
    global error_margin    
    global groups
    difference_limit_not_achieved=True
    iter=0
    while difference_limit_not_achieved:
        #New element - eta_list is calculated after each iteration for updated assignation matrix
        eta_list=calc_eta_radius(centers,assignation,data_set)
        print ('Step: '+str(iter))
        print ('Current positions of centers \n' + str(centers))
        print ('Assignation matrix: \n'+str(assignation))
        #eta_list=calc_eta_radius(centers,assignation,data_set)
        print ('Eta radius list: \n' +str(eta_list)+'\n ----')
        new_assignation=[]
        for i in range(len(data_set)):
            new_assignation_vector=[]
            for k in range(groups):
                new_assignation_vector.append(calculate_u(data_set[i],k,eta_list))
            new_assignation.append(new_assignation_vector)
        calculate_new_centers(new_assignation)
        if iter>0:
            if calculate_differences(new_assignation) < error_margin:
                difference_limit_not_achieved=False
        assignation=np.array(new_assignation)
        iter=iter+1
        if iter > 100:
            break

cluster()

Step: 0
Current positions of centers 
[array([0.0589391 , 0.15500637]), array([0.6172888 , 0.71834395])]
Assignation matrix: 
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Eta radius list: 
[0.02624460519557519, 1.4823274288211812]
 ----
Step: 1
Current positions of centers 
[array([0.05329644, 0.15157638]), array([0.43547374, 0.51164932])]
Assignation matrix: 
[[2.87566390e-02 4.40074245e-01]
 [4.78540330e-02 4.54525814e-01]
 [3.58035244e-02 4.56554000e-01]
 [5.78746301e-02 5.19564042e-01]
 [1.05669210e-02 5.69335946e-01]
 [4.91969790e-03 6.11781426e-01]
 [5.05848415e-04 6.42114878e-01]
 [1.15694696e-04 4.31335260e-01]
 [1.30792186e-04 7.67503934e-01]
 [1.09248990e-04 6.41090431e-01]]
Eta radius list: 
[0.015268828801304915, 1.3328224048245167]
 ----
Step: 2
Current positions of centers 
[array([0.03416374, 0.13659378]), array([0.26115395, 0.33810196])]
Assignation matrix: 
[[1.09676882e-02 5.73783277e-01]
 [2.30942210e-02 5.88652614e-01]
 

C:\Users\Jan\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in double_scalars
C:\Users\Jan\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Jan\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
